## Modern `dict` syntax

### `dict` Comprehensions (python3)

In [1]:
dial_codes = [
    (880, 'Bangladesh'),
    (55, 'Brazil'),
    (86, 'China'),
    (91, 'India'),
    (62, 'Indonesia'),
    (81, 'Japan'),
    (234, 'Nigeria'),
    (92, 'Pakistan'),
    (7, 'Russia'),
    (1, 'United States')
]

country_dial = {country: code for code, country in dial_codes}
country_dial

{'Bangladesh': 880,
 'Brazil': 55,
 'China': 86,
 'India': 91,
 'Indonesia': 62,
 'Japan': 81,
 'Nigeria': 234,
 'Pakistan': 92,
 'Russia': 7,
 'United States': 1}

In [2]:
{code: country.upper()
    for country, code in sorted(country_dial.items())
    if code < 70}

{55: 'BRAZIL', 62: 'INDONESIA', 7: 'RUSSIA', 1: 'UNITED STATES'}

### Unpacking Mappings (python 3.5)

In [3]:
def dump(**kwargs):
    return kwargs
dump(**{'x': 1}, y=2, **{'z': 3})

{'x': 1, 'y': 2, 'z': 3}

In [4]:
{'a': 0, **{'x': 1}, 'y': 2, **{'z': 3, 'x': 4}}

{'a': 0, 'x': 4, 'y': 2, 'z': 3}

### Merging Mappings with | (python 3.9)

In [5]:
d1 = {'a': 1, 'b': 3}
d2 = {'a': 2, 'b': 4, 'c': 6}
d1 | d2

{'a': 2, 'b': 4, 'c': 6}

In [6]:
d1

{'a': 1, 'b': 3}

In [7]:
d1 |= d2
d1

{'a': 2, 'b': 4, 'c': 6}

## Pattern Matching with Mappings
The `match/case` statement supports subjects that are mapping objects. Thanks to destructuring, *pattern matching is a powerful tool to process records structured like nested mappings and sequences, which we often need to read from JSON APIs* and databases with semi-structured schemas, like MongoDB, EdgeDB, or PostgreSQL.


In [8]:
def get_creators(record: dict) -> list:
    match record:
        case {'type': 'book', 'api': 2, 'authors': [*names]}:
            return names
        case {'type': 'book', 'api': 1, 'author': name}:
            return [name]
        case {'type': 'book'}:
            raise ValueError(f"Invalid 'book' record: {record!r}")
        case {'type': 'movie', 'director': name}:
            return [name]
        case _:
            raise ValueError(f'Invalid record: {record!r}')


In [9]:
b1 = dict(api=1, author='Douglas Hofstadter',
        type='book', title='Gödel, Escher, Bach')
get_creators(b1)

['Douglas Hofstadter']

In [12]:
from collections import OrderedDict
b2 = OrderedDict(api=2, type='book', title='Python in a Nutshell',
            authors=['Martelli', 'Ravenscroft', 'Holden'])
get_creators(b2)

['Martelli', 'Ravenscroft', 'Holden']

In [13]:
get_creators({'type': 'book', 'pages': 770})

ValueError: Invalid 'book' record: {'type': 'book', 'pages': 770}

In [14]:
food = dict(category='ice cream', flavor='vanilla', cost=199)
match food:
    case {'category': 'ice cream', **details}:
        print(f'Ice cream details: {details}')

Ice cream details: {'flavor': 'vanilla', 'cost': 199}


## Standard API of Mapping Types
The `collections.abc` module provides the `Mapping` and `MutableMapping` ABCs describing the interfaces of `dict` and similar types
<img src="../images/7.png" style="width: 70%;">

To implement a custom mapping, it’s easier to extend `collections.UserDict`, or to wrap a dict by composition, instead of subclassing these ABCs. The `collections.UserDict` class and all concrete mapping classes in the standard library encapsulate the basic dict in their implementation, which in turn is built on a hash table. Therefore, they all share the limitation that *the keys must be hashable (the values need not be hashable, only the keys)*

In [16]:
tt = (1, 2, (30, 40))
print(f"{hash(tt) = }")

hash(tt) = -3907003130834322577


In [18]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

User-defined types are hashable by default because their hash code is their `id()`, and the `__eq__()` method inherited from the object class simply compares the object IDs.  
If an object implements a custom `__eq__()` that takes into account its internal state, it will be hashable only if its `__hash__()` always returns the same hash code. In practice, this requires that `__eq__()` and `__hash__()` only take into account instance attributes that never change during the life of the object.

### Overview of Common Mapping Methods

<img src="../images/8.png" style="width: 80%;">

`my_dict.setdefault(key, []).append(new_value)` is equal to
```python
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value)
```
except that the latter code performs at least two searches for key — or even three if it’s not found — while `setdefault` does it all with a single lookup.

In [24]:
my_dict = {}
my_dict.setdefault("a", []).append(1)
print(my_dict)
my_dict.setdefault("a", []).append(2)
print(my_dict)

{'a': [1]}
{'a': [1, 2]}


## Automatic Handling of Missing Keys
Sometimes it is convenient to have mappings that return some made-up value when a missing key is searched. There are two main approaches to this: 
1. Use a `defaultdict` instead of a plain `dict`. 2
2. Subclass `dict` or any other mapping type and add a `__missing__` method

### `defaultdict`: Another Take on Missing Keys
A `collections.defaultdict` instance creates items with a default value on demand whenever a missing key is searched using `d[k]` syntax

In [36]:
import collections
from typing import DefaultDict

class A:
    def __init__(self) -> None:
        print("init A")

d: DefaultDict[str, A] = collections.defaultdict(A)
d["a1"]
print(d)

init A
defaultdict(<class '__main__.A'>, {'a1': <__main__.A object at 0x7f43212cba60>})


### The `__missing__` Method
Underlying the way mappings deal with missing keys is the aptly named `__missing__` method. This method is not defined in the base dict class, but dict is aware of it: if you subclass dict and provide a `__missing__` method, the standard `dict.__getitem__` will call it whenever a key is not found, instead of raising KeyError.

In [54]:
class StrKeyDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]  # calls __getitem__ which calls __missing__
        except KeyError:
            return default
    
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()
    
d = StrKeyDict0()
d['2'] = 'two'
d[2]

'two'

In [58]:
2 in d

True